In [ ]:
import numpy as np
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
# remove stopwords

df = pd.read_csv('USAirlinesTweets.csv', encoding='latin1')
label_mapping = {"negative": 0, "neutral": 1, "positive": 2}
news_quality = df['sentiment'].map(label_mapping)
news_text = df['tweet'].tolist()
x=[]
words=stopwords.words("english")
words+=['@','!','.',',','?']
for i in news_text:
  tweet=i
  tweet=word_tokenize(tweet.lower())
  tweet=[j for j in tweet if j not in words]
  x.append(' '.join(tweet))
x=news_text


train_news_text, test_news_text, train_news_quality, test_news_quality = train_test_split(
    news_text, news_quality, test_size=0.3, random_state=42)

encodings = tokenizer(train_news_text, truncation=True, padding=True, return_tensors="pt")
encodings_test = tokenizer(test_news_text, truncation = True, padding=True, return_tensors = "pt")
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
train_dataset = CustomDataset(encodings, torch.tensor(train_news_quality.values))
test_dataset = CustomDataset(encodings_test, torch.tensor(test_news_quality.values))
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()
    predictions = np.argmax(probs, axis=1)  # pick class with highest probability

    report = classification_report(labels, predictions, output_dict=True)
    acc = accuracy_score(labels, predictions)

    metrics = {
        "accuracy": acc,
        "f1_weighted": report["weighted avg"]["f1-score"],
        "precision_weighted": report["weighted avg"]["precision"],
        "recall_weighted": report["weighted avg"]["recall"]
    }
    for class_label in ["0", "1", "2"]:
        metrics[f"precision_class_{class_label}"] = report[class_label]["precision"]
        metrics[f"recall_class_{class_label}"] = report[class_label]["recall"]
        metrics[f"f1_class_{class_label}"] = report[class_label]["f1-score"]
        metrics[f"support_class_{class_label}"] = report[class_label]["support"]
    return metrics

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    num_train_epochs=5,
    learning_rate=1e-5
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics)
trainer.train()
trainer.evaluate()
results = trainer.evaluate()

print(results)
preds = trainer.predict(train_dataset)
preds = np.argmax(preds[:3][0],axis=1)
GT = train_news_quality
print(classification_report(GT,preds))
preds = trainer.predict(test_dataset)
preds = np.argmax(preds[:3][0],axis=1)
GT = test_news_quality
print(classification_report(GT,preds))
from transformers import TextClassificationPipeline
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


              precision    recall  f1-score   support

           0       0.98      0.98      0.98      6364
           1       0.93      0.90      0.92      2215
           2       0.93      0.95      0.94      1669

    accuracy                           0.96     10248
   macro avg       0.95      0.95      0.95     10248
weighted avg       0.96      0.96      0.96     10248



/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


              precision    recall  f1-score   support

           0       0.90      0.92      0.91      2814
           1       0.71      0.65      0.68       884
           2       0.79      0.81      0.80       694

    accuracy                           0.85      4392
   macro avg       0.80      0.79      0.80      4392
weighted avg       0.85      0.85      0.85      4392



Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
preds = trainer.predict(test_dataset)
preds = np.argmax(preds[:3][0],axis=1)
GT = test_news_quality
print(classification_report(GT,preds))

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


              precision    recall  f1-score   support

           0       0.90      0.92      0.91      2814
           1       0.71      0.65      0.68       884
           2       0.79      0.81      0.80       694

    accuracy                           0.85      4392
   macro avg       0.80      0.79      0.80      4392
weighted avg       0.85      0.85      0.85      4392



In [ ]:
# CustomDataset(encodings, torch.tensor(train_news_quality.values))
# encodings = tokenizer(train_news_text, truncation=True, padding=True, return_tensors="pt")
sentences=['I love the service that I experienced!', \
           'The customer service was horrendous and didn\'t listen to me, I demand my money back!', \
           'The workers barely put me on hold and weren\'t awful, would recommend.', \
           "Is it true that the Frequent Flyers program will be discontinued soon?", \
           "When will my flight arrive?"]
print(sentences)
x=CustomDataset(tokenizer(sentences, truncation=True, padding=True, return_tensors="pt"), torch.tensor(np.array([1]*len(sentences))))
preds2 = trainer.predict(x)
print(preds2)

['I love the service that I experienced!', "The customer service was horrendous and didn't listen to me, I demand my money back!", "The workers barely put me on hold and weren't awful, would recommend.", 'Is it true that the Frequent Flyers program will be discontinued soon?', 'When will my flight arrive?']


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


PredictionOutput(predictions=array([[-2.0549004, -1.9490422,  3.938163 ],
       [ 5.1386557, -2.089425 , -2.7402763],
       [ 4.2612185, -1.8605127, -2.227903 ],
       [ 1.8420892,  1.4021966, -3.6714497],
       [-1.5347573,  3.5893867, -2.50458  ]], dtype=float32), label_ids=array([1, 1, 1, 1, 1]), metrics={'test_loss': 4.038959503173828, 'test_accuracy': 0.2, 'test_f1_weighted': 0.3333333333333333, 'test_precision_weighted': 1.0, 'test_recall_weighted': 0.2, 'test_precision_class_0': 0.0, 'test_recall_class_0': 0.0, 'test_f1_class_0': 0.0, 'test_support_class_0': 0.0, 'test_precision_class_1': 1.0, 'test_recall_class_1': 0.2, 'test_f1_class_1': 0.3333333333333333, 'test_support_class_1': 5.0, 'test_precision_class_2': 0.0, 'test_recall_class_2': 0.0, 'test_f1_class_2': 0.0, 'test_support_class_2': 0.0, 'test_runtime': 0.4965, 'test_samples_per_second': 10.07, 'test_steps_per_second': 2.014})


In [ ]:
train_news_text[0]

'@JetBlue Re: Flight 8088 SXM&gt;JFK what time does bus leave hotel on Feb 22?  Mass confusion here.'

In [ ]:
df = pd.read_csv('USAirlinesTweets.csv', encoding='latin1')
label_mapping = {"negative": 0, "neutral": 1, "positive": 2}
news_quality = df['sentiment'].map(label_mapping)
news_text = df['tweet'].tolist()
train_news_text, test_news_text, train_news_quality, test_news_quality = train_test_split(
    news_text, news_quality, test_size=0.3, random_state=42)

In [ ]:
trainer.push_to_hub()

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-6874afae-530e1124620f0e4870d3f8b8;1dca2e52-725b-4005-97c1-7b2e63f3f72f)

Invalid username or password.

,sentiment,negativereason,tweet
0,neutral,NaN,@VirginAmerica What @dhepburn said.
1,positive,NaN,@VirginAmerica plus you've added commercials t...
2,neutral,NaN,@VirginAmerica I didn't today... Must mean I n...
3,negative,Bad Flight,@VirginAmerica it's really aggressive to blast...
4,negative,Can't Tell,@VirginAmerica and it's a really big bad thing...
...,...,...,...
995,negative,Late Flight,@united by the time I finally get to Dallas I ...
996,negative,Late Flight,@united I'm trying to get to my final destinat...
997,negative,Customer Service Issue,@united that guy really has no customer servic...
998,positive,NaN,@united he has no priority and Iove it
